<a ><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Logo_INSA_Lyon_%282014%29.svg/langfr-2560px-Logo_INSA_Lyon_%282014%29.svg.png"  width="200" align="left"> </a>
<div style="text-align: right"> <h3><span style="color:gray"> Projet de recherche </span> </h3> </div>

<br>
<br>
<br>


<h1><center>Data Merge</center></h1>
<h2><center> <span style="font-weight:normal"><font color='#e42618'> Combining Analyst Ratings with Market Data</font>  </span></center></h2>


<h3><center><font color='gray'>JONAS GOTTAL</font></center></h3>





<h4>Project scope</h4>

- Obtaining financial research containing both a report in natural language and a quantifiable prediction on the underlying asset
- Building a predictive models that e.g., accurately detect semantic causality
- Evaluation whether there is a correlation between causal formulations and a higher accuracy from an analyst
<br>

---
---

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="text-decoration:none; margin-top: 30px; background-color:#F2F2F2; border-color:#720006">
    <span style="color:#720006">
    <ol>
        <li><a href="#1"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Exploratory Analysis: Ticker Symbols</span> </a></li>
       <li><a href="#2"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Structured Approach</span> </a></li> 
       <ol>
       <li><a href="#3"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Minimum working example</span> </a></li>
       <li><a href="#4"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Execution in batches (API)</span> </a></li>
        <li><a href="#5"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Additional verification</span> </a></li>
       </ol>
       <li><a href="#6"> <span style="color:#720006;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Merging of Market Data based on Symbols</span> </a></li>
    </ol>
    </span>
</div>

#### Requirements
- ```Python 3.9.18``` (conda env)
- ```pip freeze > requirements.txt```
- ```conda env export > environment.yml```

# Exploratory Analysis: Ticker Symbols <a id="1"></a>

- Problem: Ticker symbols are incomplete. E.g.: "Central Japan Railway Co" has Ticker ```9022```, but is only callable by ```9022.T``` (traded at Tokyo exchange)
- Our approach: finding the underlying Ticker symbol based on the names (which are definitely clean)
- We do this by scraping Yahoo Finance search results and select the most likely result based on:
    - Highest traded volume (meaning if others think this is *the* company, so do we) 
    - Matching currency

This works, as yahoo finance only returns results that definitely fit. The additional criteria are necessary since the same stock can be traded at multiple exchanges and also OTC (over the counter, not public). With the additional criteria and some cleaning of the names and currencies we can achieve a high success rate.

# Structured Approach <a id="2"></a>

In [1]:
# load the df from csv
import pandas as pd
import requests
import yfinance as yf
import numpy as np
import time
from retrying import retry

df = pd.read_csv('Data/data.csv', index_col=0)

### Data cleaning

In [2]:
# replace GBX in the currency column with GBP
df["Currency"] = df["Currency"].replace("GBX", "GBP")
df["Currency"].unique()

array(['GBP', 'AUD', 'EUR', 'USD', 'HKD', 'CNY', 'NZD', 'CAD', 'JPY',
       'CHF', 'NOK', 'INR', 'PLN', 'SGD', 'DKK', 'TWD', 'KRW', 'SAR',
       'SEK', 'BRL', 'IDR'], dtype=object)

## Minimal working example <a id="3"></a>

### Obtain List of potential ticker symbols

In [3]:
def get_ticker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 5}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()
    # give all symbols
    company_codes = []
    for i in data['quotes']:
        if i["isYahooFinance"]==True:
            company_codes.append(i['symbol'])
    return company_codes

### Check based on Volume and currency which is the most likely

In [4]:
#loop over list and return for highest value
def select_ticker(company_name, currency):
    ticker = get_ticker(company_name)
    highest = 0
    tickername = ""
    for i in ticker:
        try:
            if yf.Ticker(i).info["volume"] > highest and currency == yf.Ticker(i).info["currency"].upper(): #currency is in "GBp" format-> upper() to match
                # tested for all companies
                highest = yf.Ticker(i).info["volume"]
                tickername = i
        except:
            pass
    return tickername   

## Execution in Batches <a id="4"></a>

Unfortunately the Yahoo Finance API is incredibly unstable. This makes scraping very hard. Thus we opt for batch processing and save all batches each as CSV files.

In [5]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [171]:

for x in batch(range(len(df)), 50):
    ticker_list = []
    for i in x:
        # skip old rows
        if i >=0:
            name = select_ticker(df.iloc[i]['CompanyName'].replace("Ltd", "").replace("(The)", ""), df.iloc[i]['Currency'])
            if name == "":
                name = df.iloc[i]['TickerSymbol'] # fallback to original ticker symbol

            ticker_list.append(name)
            time.sleep(2.5)

            # print progress
            print(f"{i+1}/{len(df)}")
        # save every 50 rows to csv file
    #if ticker list is not empty
    if ticker_list != []:
        pd.DataFrame(ticker_list).to_csv(f"TickerLists/ticker_list_{x[0]}_{x[-1]}.csv") 
        time.sleep(90) #sleep for 1 minute to avoid being blocked by yahoo finance
    

1551/1626
1552/1626
1553/1626
1554/1626
1555/1626
1556/1626
1557/1626
1558/1626
1559/1626
1560/1626
1561/1626
1562/1626
1563/1626
1564/1626
1565/1626
1566/1626
1567/1626
1568/1626
1569/1626
1570/1626
1571/1626
1572/1626
1573/1626
1574/1626
1575/1626
1576/1626
1577/1626
1578/1626
1579/1626
1580/1626
1581/1626
1582/1626
1583/1626
1584/1626
1585/1626
1586/1626
1587/1626
1588/1626
1589/1626
1590/1626
1591/1626
1592/1626
1593/1626
1594/1626
1595/1626
1596/1626
1597/1626
1598/1626
1599/1626
1600/1626
1601/1626
1602/1626
1603/1626
1604/1626
1605/1626
1606/1626
1607/1626
1608/1626
1609/1626
1610/1626
1611/1626
1612/1626
1613/1626
1614/1626
1615/1626
1616/1626
1617/1626
1618/1626
1619/1626
1620/1626
1621/1626
1622/1626
1623/1626
1624/1626
1625/1626
1626/1626


In [6]:
sym_list = []
for x in batch(range(len(df)), 50):
    #load csv files
    ticker_list = pd.read_csv(f"Data/TickerLists/ticker_list_{x[0]}_{x[-1]}.csv", index_col=0)
        
    # concat all ticker lists
    sym_list.extend(ticker_list.values.tolist())
        
sym_list

# flatten the list of lists
flat_list = [item for sublist in sym_list for item in sublist]
flat_list

['VMUK.L',
 'VUK.AX',
 'QUB.AX',
 '0N9G.IL',
 'SGE.L',
 'CS.PA',
 'AXAHY',
 'CRH',
 'KGN.AX',
 '0780.HK',
 'BSL.AX',
 'GDS',
 '9698.HK',
 'WEB.AX',
 'DE',
 'AVGO',
 'AME',
 'DOV',
 'HUBB',
 'ITT',
 'JBT',
 'MAN',
 'RHI',
 'ROP',
 'DTE',
 'ENEL.MI',
 'NJR',
 'AEE',
 'HD',
 'JMAT.L',
 'IQ',
 'KGF.L',
 'KGFHY',
 '9888.HK',
 'BIDU',
 '1024.HK',
 'URBN',
 '600905.SS',
 'IMAX',
 'VIV.PA',
 'VIVEF',
 'JWN',
 'ADSK',
 'HPQ',
 'J',
 'NVDA',
 'AX1.AX',
 'EBO.NZ',
 'TW',
 'MPL.AX',
 'CFR',
 'CFG',
 'HBAN',
 'EXC',
 'GTII.CN',
 'HUB.AX',
 '9022.T',
 'WN-PC.TO',
 'MDT',
 'BBY',
 'LOW',
 'KSS',
 'ADI',
 'SOON.SW',
 'ADNT',
 'BAYN.DE',
 'BAYRY',
 'FICO',
 'WM',
 '7974.T',
 'TNE.AX',
 '9961.HK',
 'TCOM',
 '600519.SS',
 '1876.HK',
 'ZM',
 'NEM',
 'NEM',
 'KEYS',
 'AVH.AX',
 'REH.AX',
 '9020.T',
 'ACL.AX',
 'HLS.AX',
 'A',
 'QCOM',
 'NRG',
 'LW',
 'CTD.AX',
 'MS',
 'GS',
 '0A9E.IL',
 '1810.HK',
 'LONN.SW',
 'GOOGL',
 'GOOGL',
 'MELI',
 'DAVA',
 'GLOBD.BA',
 'AHT.L',
 'ALO.PA',
 '6758.T',
 'SONY',
 '6976

In [7]:
df["FinalTickerSymbol"] = flat_list

In [8]:
df

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,BullsList,BearsList,ResearchThesisDate,ResearchThesisList,MoatAnalysis,RiskAnalysis,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol
0,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VMUK,4,24/11/2023,Nathan Zaia,146.75,GBP,23/11/2023,...,The combination of two midtier banks should de...,Customers placing less emphasis on local branc...,24/11/2023,Virgin Money UK (LSE: VMUK; ASX: VUK) consists...,"Virgin Money UK, or Virgin, is the largest mid...",Banks are highly leveraged to economic conditi...,Our capital allocation rating for Virgin Money...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VMUK.L
1,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VUK,4,24/11/2023,Nathan Zaia,2.78,AUD,24/11/2023,...,The combination of two midtier banks should de...,Customers placing less emphasis on local branc...,24/11/2023,Virgin Money UK (LSE: VMUK; ASX: VUK) consists...,"Virgin Money UK, or Virgin, is the largest mid...",Banks are highly leveraged to economic conditi...,Our capital allocation rating for Virgin Money...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VUK.AX
2,24/11/2023,Qube's Good Momentum Likely to Carry Into Fisc...,Qube Holdings Ltd,QUB,4,24/11/2023,Adrian Atkins,2.93,AUD,24/11/2023,...,There is significant potential to increase eff...,"A corporate structure of associate companies, ...",24/11/2023,Qube's strategy is to consolidate the fragment...,Qube has established an integrated ports and l...,We assign Qube a Medium Morningstar Uncertaint...,We assign Qube Holdings an Exemplary Capital A...,Qube has three main divisions: operating; prop...,Qube is in strong financial health. Gearing (n...,QUB.AX
3,24/11/2023,Stabilization of Political Risk in Spain Is Po...,Endesa SA,ELE,4,23/11/2023,Tancrede Fulop,18.84,EUR,24/11/2023,...,Increasing renewable capacity should improve t...,The lack of an international footprint implies...,26/05/2023,Endesa is an integrated utility operating in p...,We think Endesa lacks an economic moat. Owing ...,We assign Endesa a Medium Uncertainty Rating.E...,We rate Endesa's capital allocation as Standar...,"Endesa generates, distributes, and supplies el...",We expect Endesa's net debt/EBITDA ratio to de...,0N9G.IL
4,24/11/2023,Sage Has Reached an Inflection Point in Its Cl...,Sage Group (The) PLC,SGE,1,23/11/2023,Rob Hales,1130.50,GBP,23/11/2023,...,Sage is over the hump in its transition to a p...,The market for financial management software f...,23/11/2023,Sage Group is in the later stages of its trans...,We think Sage has a narrow economic moat based...,We assign Sage a Medium Morningstar Uncertaint...,We assign Sage a Standard Morningstar Capital ...,The Sage Group is a U.K.-based provider of acc...,Sage is in good financial condition. Our model...,SGE.L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,25/11/2023,Minimal Changes To Our Fair Value After Fiscal...,The Toronto-Dominion Bank,TD,4,NaN,Eric Compton,83.15,CAD,23/11/2023,...,The profitability of its Canadian bank segment...,The bank's higher U.S. exposure and higher rat...,28/08/2023,Toronto-Dominion is one of the two largest ban...,"With its solid funding base, leading share in ...",Canadian banks face two primary risks: macroec...,We give Toronto-Dominion an Exemplary capital ...,Toronto-Dominion is one of Canada's two larges...,We think Toronto-Dominion is in strong overall...,TD.TO
1622,25/11/2023,Xiabuxiabu's Value-Oriented Model Should Fare ...,Xiabuxiabu Catering Management (China) Hldgs C...,00520,4,28/08/2023,Ivan Su,3.25,HKD,24/11/2023,...,Value-oriented operations like the Xiabuxiabu ...,"Although the zero-COVID-19 policy is now over,...",28/08/2023,"Xiabuxiabus multiconcept portfolio, operating 

## Additional verification <a id="5"></a>

Now we merge our actual market data with our reports. By doing so, we also add important information like market cap, sectors and traded volume. For the price data, we also check whether our retrieved ticker symbols dedicated price match for the same date (within some fluctuation). And then add current prices in 5, 10, 30, 90 day steps.

In [11]:
# def that returns the sector, market cap, exchange and volume based on ticker symbol

def get_info(ticker):
    try:
        ticker = yf.Ticker(ticker)
        sector = ticker.info["sector"]
        market_cap = ticker.info["marketCap"]
        exchange = ticker.info["exchange"]
        volume = ticker.info["regularMarketVolume"]
        currency = ticker.info["currency"]
        return sector, market_cap, exchange, volume, currency
    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    

# based on column "Symbol", get the sector, market cap, exchange and volume and add it to the df
sector_list = []    
market_cap_list = []
exchange_list = []
volume_list = []
currency_list = []
n=0
for i in df["FinalTickerSymbol"]:
    sector, market_cap, exchange, volume, currency = get_info(i)
    sector_list.append(sector)
    market_cap_list.append(market_cap)
    exchange_list.append(exchange)
    volume_list.append(volume)
    currency_list.append(currency)
    # print progress
    print(f"{n+1}/{len(df)}")
    n+=1


df["Sector"] = sector_list
df["MarketCap"] = market_cap_list
df["Exchange"] = exchange_list
df["Volume"] = volume_list
df["TickerCurrency"] = currency_list

1/1626
2/1626
3/1626
4/1626
5/1626
6/1626
7/1626
8/1626
9/1626
10/1626
11/1626
12/1626
13/1626
14/1626
15/1626
16/1626
17/1626
18/1626
19/1626
20/1626
21/1626
22/1626
23/1626
24/1626
25/1626
26/1626
27/1626
28/1626
29/1626
30/1626
31/1626
32/1626
33/1626
34/1626
35/1626
36/1626
37/1626
38/1626
39/1626
40/1626
41/1626
42/1626
43/1626
44/1626
45/1626
46/1626
47/1626
48/1626
49/1626
50/1626
51/1626
52/1626
53/1626
54/1626
55/1626
56/1626
57/1626
58/1626
59/1626
60/1626
61/1626
62/1626
63/1626
64/1626
65/1626
66/1626
67/1626
68/1626
69/1626
70/1626
71/1626
72/1626
73/1626
74/1626
75/1626
76/1626
77/1626
78/1626
79/1626
80/1626
81/1626
82/1626
83/1626
84/1626
85/1626
86/1626
87/1626
88/1626
89/1626
90/1626
91/1626
92/1626
93/1626
94/1626
95/1626
96/1626
97/1626
98/1626
99/1626
100/1626
101/1626
102/1626
103/1626
104/1626
105/1626
106/1626
107/1626
108/1626
109/1626
110/1626
111/1626
112/1626
113/1626
114/1626
115/1626
116/1626
117/1626
118/1626
119/1626
120/1626
121/1626
122/1626
123/1626
1

In [12]:
df

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,RiskAnalysis,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency
0,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VMUK,4,24/11/2023,Nathan Zaia,146.75,GBP,23/11/2023,...,Banks are highly leveraged to economic conditi...,Our capital allocation rating for Virgin Money...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VMUK.L,Financial Services,1.932661e+09,LSE,45564.0,GBp
1,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VUK,4,24/11/2023,Nathan Zaia,2.78,AUD,24/11/2023,...,Banks are highly leveraged to economic conditi...,Our capital allocation rating for Virgin Money...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VUK.AX,Financial Services,3.709752e+09,ASX,2057263.0,AUD
2,24/11/2023,Qube's Good Momentum Likely to Carry Into Fisc...,Qube Holdings Ltd,QUB,4,24/11/2023,Adrian Atkins,2.93,AUD,24/11/2023,...,We assign Qube a Medium Morningstar Uncertaint...,We assign Qube Holdings an Exemplary Capital A...,Qube has three main divisions: operating; prop...,Qube is in strong financial health. Gearing (n...,QUB.AX,Industrials,5.229462e+09,ASX,2520549.0,AUD
3,24/11/2023,Stabilization of Political Risk in Spain Is Po...,Endesa SA,ELE,4,23/11/2023,Tancrede Fulop,18.84,EUR,24/11/2023,...,We assign Endesa a Medium Uncertainty Rating.E...,We rate Endesa's capital allocation as Standar...,"Endesa generates, distributes, and supplies el...",We expect Endesa's net debt/EBITDA ratio to de...,0N9G.IL,Utilities,2.128164e+10,IOB,5007.0,EUR
4,24/11/2023,Sage Has Reached an Inflection Point in Its Cl...,Sage Group (The) PLC,SGE,1,23/11/2023,Rob Hales,1130.50,GBP,23/11/2023,...,We assign Sage a Medium Morningstar Uncertaint...,We assign Sage a Standard Morningstar Capital ...,The Sage Group is a U.K.-based provider of acc...,Sage is in good financial condition. Our model...,SGE.L,Technology,1.172798e+10,LSE,253230.0,GBp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,25/11/2023,Minimal Changes To Our Fair Value After Fiscal...,The Toronto-Dominion Bank,TD,4,NaN,Eric Compton,83.15,CAD,23/11/2023,...,Canadian banks face two primary risks: macroec...,We give Toronto-Dominion an Exemplary capital ...,Toronto-Dominion is one of Canada's two larges...,We think Toronto-Dominion is in strong overall...,TD.TO,Financial Services,1.466779e+11,TOR,4963474.0,CAD
1622,25/11/2023,Xiabuxiabu's Value-Oriented Model Should Fare ...,Xiabuxiabu Catering Management (China) Hldgs C...,00520,4,28/08/2023,Ivan Su,3.25,HKD,24/11/2023,...,Our Morningstar Uncertainty Rating for Xiabuxi...,"Overall, we rate Xiabuxiabus Morningstar Capit...",Xiabuxiabu Catering Management is a Chinese mu...,Xiabuxiabu holds a strong financial position w...,00520,NaN,NaN,NaN,NaN,NaN
1623,25/11/2023,NextDC's Australia Wide Data Center Network Ex...,Nextdc Ltd,NXT,3,28/08/2023,Dan Baker,12.92,AUD,24/11/2023,...,Our Morningstar Uncertainty Rating for NextDC ...,We assign NextDC a Standard Capital Allocation...,NextDC operates 13 data centers across five ma...,"NextDC is in sound financial health, but we ex...",NXT.AX,Technology,6.918949e+09,ASX,1956904.0,AUD
1624,25/11/2023,EVT on Track To Return to Prepandemic Earnings,EVT Ltd,EVT,4,28/08/2023,Brian Han,10.82,AUD,24/11/2023,...,We assign EVT a Morningstar Uncertainty Rating...,Our Morningstar Capital Allocation Rating for ...,EVT is an Australian provider of entertainment...,EVT has the liquidity to see through the curre...,EVT,Financial Services,1.573599e+09,NYQ,194060.0,USD


First we try to improve the results based on the uneven currencies. Meaning if it should be AUD for example, but we only found the stock in USD, we add ".AX" to the ticker and re-try it

In [11]:
# we only check for USD because this is the default exchange location for yahoo finance
df[(df["Currency"]!="USD") & (df["TickerCurrency"]=="USD")]["Currency"].unique()

array(['AUD'], dtype=object)

In [10]:
# Show all rows where the Symbol has a space in it
df[df["FinalTickerSymbol"].str.contains(" ")]
# replace the space with a minus
df["FinalTickerSymbol"] = df["FinalTickerSymbol"].str.replace(" ", "-")

In [13]:
# show all rows where the currency is USD and the ticker currency is AUD
# for those we add ".AX" to the ticker symbol in the FinalTickerSymbol column
df.loc[(df["Currency"]=="AUD") & (df["TickerCurrency"]=="USD"), "FinalTickerSymbol"] = df["FinalTickerSymbol"] + ".AX"
df[(df["Currency"]=="AUD") & (df["TickerCurrency"]=="USD")]



,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,RiskAnalysis,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency
77,24/11/2023,Retaining Newmont's FVE at USD 53 Per Share; S...,Newmont Corp,NEM,5,21/11/2023,Jon Mills,57.10,AUD,24/11/2023,...,"Our Uncertainty Rating for Newmont is Medium, ...",Our capital allocation rating for Newmont is S...,Newmont is the world's largest gold miner. It ...,"In our view, Newmont is in sound financial hea...",NEM.AX,Basic Materials,4.748259e+10,NYQ,13861459.0,USD
180,24/11/2023,Longer-Term Margin Recovery Still Likely for AMP,AMP Ltd,AMP,4,16/11/2023,Shaun Ler,0.91,AUD,24/11/2023,...,We assign AMP a High Morningstar Uncertainty R...,We assign AMP a Poor Morningstar Capital Alloc...,"At its roots, AMP is a wealth manager, providi...",AMPs financial position is sound. AMP has cons...,AMP.AX,Financial Services,3.605512e+10,NYQ,667482.0,USD
302,24/11/2023,Shares in APM Human Services Appear Undervalue...,APM Human Services International Pty,APM,5,13/11/2023,Shane Ponraj,1.48,AUD,24/11/2023,...,We assign a APM a Morningstar Uncertainty Rati...,We assign APM a Standard Morningstar Capital A...,"APM Human Services International, on behalf of...",APM is in a fairly comfortable financial posit...,APM.AX,Healthcare,8.705256e+06,NCM,13415.0,USD
797,24/11/2023,CSR Earnings: Strong Backlog Shielding Sales F...,CSR Ltd,CSR,3,02/11/2023,Brian Han,5.86,AUD,24/11/2023,...,We assign CSR a Medium Morningstar Uncertainty...,We rate CSRs Capital Allocation as Standard. O...,CSR is one of Australias leading building mate...,CSR generally has a conservative balance sheet...,CSR.AX,Real Estate,8.721299e+08,NYQ,124940.0,USD
1500,25/11/2023,Housing Construction to Slow Near-Term but Bri...,Washington H Soul Pattinson & Co Ltd,SOL,2,29/09/2023,Johannes Faul,33.05,AUD,24/11/2023,...,"We assign Washington H Soul Pattinson, or WHSP...",We assign WHSP with a Standard capital allocat...,"Washington H. Soul Pattinson, or WHSP, is a va...",WHSP has an appropriately conservative approac...,SOL.AX,Technology,1.472135e+08,NYQ,539195.0,USD
1624,25/11/2023,EVT on Track To Return to Prepandemic Earnings,EVT Ltd,EVT,4,28/08/2023,Brian Han,10.82,AUD,24/11/2023,...,We assign EVT a Morningstar Uncertainty Rating...,Our Morningstar Capital Allocation Rating for ...,EVT is an Australian provider of entertainment...,EVT has the liquidity to see through the curre...,EVT.AX,Financial Services,1.573599e+09,NYQ,194060.0,USD


In [14]:
# we update the sector, market cap, exchange and volume based on the new ticker symbol
# based on column "Symbol", get the sector, market cap, exchange and volume and add it to the df
sector_list = []    
market_cap_list = []
exchange_list = []
volume_list = []
currency_list = []
n=0

for i in df["FinalTickerSymbol"]:
    sector, market_cap, exchange, volume, currency = get_info(i)
    sector_list.append(sector)
    market_cap_list.append(market_cap)
    exchange_list.append(exchange)
    volume_list.append(volume)
    currency_list.append(currency)
    # print progress
    print(f"{n+1}/{len(df)}")
    n+=1


df["Sector"] = sector_list
df["MarketCap"] = market_cap_list
df["Exchange"] = exchange_list
df["Volume"] = volume_list
df["TickerCurrency"] = currency_list

1/1626
2/1626
3/1626
4/1626
5/1626
6/1626
7/1626
8/1626
9/1626
10/1626
11/1626
12/1626
13/1626
14/1626
15/1626
16/1626
17/1626
18/1626
19/1626
20/1626
21/1626
22/1626
23/1626
24/1626
25/1626
26/1626
27/1626
28/1626
29/1626
30/1626
31/1626
32/1626
33/1626
34/1626
35/1626
36/1626
37/1626
38/1626
39/1626
40/1626
41/1626
42/1626
43/1626
44/1626
45/1626
46/1626
47/1626
48/1626
49/1626
50/1626
51/1626
52/1626
53/1626
54/1626
55/1626
56/1626
57/1626
58/1626
59/1626
60/1626
61/1626
62/1626
63/1626
64/1626
65/1626
66/1626
67/1626
68/1626
69/1626
70/1626
71/1626
72/1626
73/1626
74/1626
75/1626
76/1626
77/1626
78/1626
79/1626
80/1626
81/1626
82/1626
83/1626
84/1626
85/1626
86/1626
87/1626
88/1626
89/1626
90/1626
91/1626
92/1626
93/1626
94/1626
95/1626
96/1626
97/1626
98/1626
99/1626
100/1626
101/1626
102/1626
103/1626
104/1626
105/1626
106/1626
107/1626
108/1626
109/1626
110/1626
111/1626
112/1626
113/1626
114/1626
115/1626
116/1626
117/1626
118/1626
119/1626
120/1626
121/1626
122/1626
123/1626
1

In [16]:
df[(df["Currency"]!="USD") & (df["TickerCurrency"]=="USD")]

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,RiskAnalysis,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency


# Merge with market data  <a id="6"></a>

For each of the newly found ticker symbols, we then check wheather the price is close enough to the initially listed prices in the report

In [17]:
@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_market_data(date, symbol):
    start = date[2]+"-"+date[1]+"-"+(str(int(date[0])-1))
    end = date[2]+"-"+date[1]+"-"+(str(int(date[0])+1))
    price = yf.download(symbol, start=start, end=end, period='1d')["Adj Close"].values[0]
    return price


In [18]:
ticker_prices = []
price = 0
# we loop over i elements in the df
for i in range(len(df)):
    # we split the date into day, month and year
    date = df["PriceDate"][i].split("/")
    # format as YYYY-MM-DD with datetime
    symbol = df["FinalTickerSymbol"][i]
    try:
        # get the price for the ticker symbol
        price = get_market_data(date, symbol)
    except:
        price = np.nan
        
    ticker_prices.append(price)
    # print progress
    print(f"{i+1}/{len(df)}")

df["TickerPrice"] = ticker_prices

[*********************100%%**********************]  1 of 1 completed
1/1626
[*********************100%%**********************]  1 of 1 completed
2/1626
[*********************100%%**********************]  1 of 1 completed
3/1626
[*********************100%%**********************]  1 of 1 completed
4/1626
[*********************100%%**********************]  1 of 1 completed
5/1626
[*********************100%%**********************]  1 of 1 completed
6/1626
[*********************100%%**********************]  1 of 1 completed
7/1626
[*********************100%%**********************]  1 of 1 completed
8/1626
[*********************100%%**********************]  1 of 1 completed
9/1626
[*********************100%%**********************]  1 of 1 completed
10/1626
[*********************100%%**********************]  1 of 1 completed
11/1626
[*********************100%%**********************]  1 of 1 completed
12/1626
[*********************100%%**********************]  1 of 1 completed
13/1626
[*******


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-22 -> 2023-11-24)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-22 -> 2023-11-24)')



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['00520']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['00520']: Exception('%ticker%: No timezone found, symbol may be delisted')



1623/1626
[*********************100%%**********************]  1 of 1 completed
1624/1626
[*********************100%%**********************]  1 of 1 completed
1625/1626
[*********************100%%**********************]  1 of 1 completed
1626/1626


In [19]:
# show all rows where TickerPrice is NaN
df[df["TickerPrice"].isna()]

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice
107,24/11/2023,No-Moat Snam to See Higher Allowed Returns in ...,Snam SpA,SRG,3,20/11/2023,Tancrede Fulop,4.54,EUR,23/11/2023,...,"Based on our capital allocation framework, whi...","Snam operates a network of about 38,000 kilome...",We project net debt to increase from EUR 11.9 ...,SRGM.XD,NaN,NaN,NaN,NaN,NaN,NaN
177,24/11/2023,Allfunds: The Largest European Fund Platform,Allfunds Group PLC,ALLFG,4,16/11/2023,Niklas Kammer,5.83,EUR,23/11/2023,...,We rate Allfunds capital allocation as Standar...,Allfunds is a fund distribution platform conne...,Allfunds is in good financial health. As of th...,ALLFGA.XD,NaN,NaN,NaN,NaN,NaN,NaN
247,24/11/2023,Resilient Performance from the Minerals Segmen...,ALS Ltd,ALQ,1,14/11/2023,Mark Taylor,12.20,AUD,24/11/2023,...,Our capital allocation rating for ALS is Stand...,Founded in the 1880s and listing on the ASX in...,ALS is in reasonable financial health. Increme...,ALQ,NaN,NaN,NaN,NaN,NaN,NaN
277,24/11/2023,The Pandemic Has Made Samsonite a More Efficie...,Samsonite International SA,01910,3,14/11/2023,Ivan Su,23.95,HKD,24/11/2023,...,We assign Samsonite a Standard Capital Allocat...,Samsonite is the worlds largest luggage compan...,Samsonite's financial health was weakened by t...,01910,NaN,NaN,NaN,NaN,NaN,NaN
346,24/11/2023,Richemont's Jewellery Maisons Continue To Outp...,Compagnie Financiere Richemont SA,CFR,4,10/11/2023,Jelena Sokolova,112.10,CHF,24/11/2023,...,We consider Richemont's Morningstar Capital Al...,Richemont is a luxury goods conglomerate with ...,Richemont's net cash position was EUR 6.6 bill...,CFRAO.SW,NaN,NaN,NaN,NaN,NaN,NaN
349,24/11/2023,"Credit Agricole Is a Unique, Diversified Bank ...",Credit Agricole SA,ACA,4,10/11/2023,Johann Scholtz,11.70,EUR,23/11/2023,...,We rate Credit Agricole S.A.'s capital allocat...,Credit Agricole S.A. is majority-owned by a gr...,We estimate the average fully loaded common eq...,ACAP.XD,NaN,NaN,NaN,NaN,NaN,NaN
413,24/11/2023,ABN Amro Well Positioned in Oligopolistic Dutc...,ABN AMRO Bank NV,ABN,5,09/11/2023,Johann Scholtz,12.38,EUR,23/11/2023,...,We give ABN Amro a Standard Capital Allocation...,"ABN Amro Bank is a Dutch bank, and the Netherl...",Even after taking into considerations the more...,ABNA.XD,NaN,NaN,NaN,NaN,NaN,NaN
425,24/11/2023,Arkema Is on Track To Become a 100% Specialty ...,Arkema SA,AKE,4,09/11/2023,Rob Hales,91.06,EUR,23/11/2023,...,We assign Arkema a Standard Morningstar Capita...,Arkema is a French chemical company formerly k...,Arkema is in good financial health. Our model-...,AKEP.XD,NaN,NaN,NaN,NaN,NaN,NaN
433,24/11/2023,Nexi's Q1 Lines up With Expectations But We Lo...,Nexi SpA,NEXI,5,09/11/2023,Niklas Kammer,6.96,EUR,23/11/2023,...,We assign Nexi a Standard capital allocation r...,Nexi is a payment services provider offering m...,We anticipate that Nexi will be able to bring ...,NEXIM.XD,NaN,NaN,NaN,NaN,NaN,NaN
506,24/11/2023,Pandora Expects Sales Acceleration in Future,Pandora A/S,PNDORA,3,08/11/2023,Jelena Sokolova,911.40,DKK,24/11/2023,...,We give Pandora management a Standard Mornings...,Pandora is a premium jewelry brand and the mar...,We believe Pandora is in reasonably good finan...,PNDORC.XD,NaN,NaN,NaN,NaN,NaN,NaN


### For HK, JP, .. we can try similar to Australia:


In [20]:
# for all rows where TickerPrice is NaN, we try to get the price again
# if currency == HKD, we remove the first character from the ticker symbol in the FinalTickerSymbol column and add ".HK" to the end
# if currency == JPY, we add ".T" to the ticker symbol in the FinalTickerSymbol column
# if currency == AUD, we add ".AX" to the ticker symbol in the FinalTickerSymbol column
# if currency == CNY, we add ".SZ" to the ticker symbol in the FinalTickerSymbol column
# if currency == CFH, we add ".SW" to the ticker symbol in the FinalTickerSymbol column
# if currency == GBP, we add ".L" to the ticker symbol in the FinalTickerSymbol column
# if currency == DKK, we add ".CO" to the ticker symbol in the FinalTickerSymbol column
# we then try to get the price again

for i in df[df["TickerPrice"].isna()].index:
    if df["Currency"][i] == "HKD":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i][1:] + ".HK"
    elif df["Currency"][i] == "JPY":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".T"
    elif df["Currency"][i] == "AUD":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".AX"
    elif df["Currency"][i] == "CNY":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".SZ"
    elif df["Currency"][i] == "CHF":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".SW"
    elif df["Currency"][i] == "GBP":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".L"
    elif df["Currency"][i] == "DKK":
        df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".CO"
    
    else:
        pass


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46238/4085828163.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["FinalTickerSymbol"][i] = df["TickerSymbol"][i] + ".AX"
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46238/4085828163.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["FinalTickerSymbol"][i] = df["TickerSymbol"][i][1:] + ".HK"
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46238/4085828163.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [21]:
price = 0
# we loop over i elements in the df
for i in df[df["TickerPrice"].isna()].index:
    # we split the date into day, month and year
    date = df["PriceDate"][i].split("/")
    # format as YYYY-MM-DD with datetime
    symbol = df["FinalTickerSymbol"][i]
    try:
        # get the price for the ticker symbol
        price = get_market_data(date, symbol)
    except:
        price = np.nan
        
    df["TickerPrice"][i] = price
    # print progress

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-22 -> 2023-11-24)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-22 -> 2023-11-24)')



[*********************100%%**********************]  1 of 1 completed

...


1 Failed download:
['ITXE.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-23 -> 2023-11-25)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['ITXE.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-23 -> 2023-11-25)')



[*********************100%%**********************]  1 of 1 completed


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46238/3356140720.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TickerPrice"][i] = price
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46238/3356140720.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TickerPrice"][i] = price


In [22]:
df[df["TickerPrice"].isna()]

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,CapitalAllocation,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice
107,24/11/2023,No-Moat Snam to See Higher Allowed Returns in ...,Snam SpA,SRG,3,20/11/2023,Tancrede Fulop,4.54,EUR,23/11/2023,...,"Based on our capital allocation framework, whi...","Snam operates a network of about 38,000 kilome...",We project net debt to increase from EUR 11.9 ...,SRGM.XD,NaN,NaN,NaN,NaN,NaN,NaN
177,24/11/2023,Allfunds: The Largest European Fund Platform,Allfunds Group PLC,ALLFG,4,16/11/2023,Niklas Kammer,5.83,EUR,23/11/2023,...,We rate Allfunds capital allocation as Standar...,Allfunds is a fund distribution platform conne...,Allfunds is in good financial health. As of th...,ALLFGA.XD,NaN,NaN,NaN,NaN,NaN,NaN
349,24/11/2023,"Credit Agricole Is a Unique, Diversified Bank ...",Credit Agricole SA,ACA,4,10/11/2023,Johann Scholtz,11.70,EUR,23/11/2023,...,We rate Credit Agricole S.A.'s capital allocat...,Credit Agricole S.A. is majority-owned by a gr...,We estimate the average fully loaded common eq...,ACAP.XD,NaN,NaN,NaN,NaN,NaN,NaN
413,24/11/2023,ABN Amro Well Positioned in Oligopolistic Dutc...,ABN AMRO Bank NV,ABN,5,09/11/2023,Johann Scholtz,12.38,EUR,23/11/2023,...,We give ABN Amro a Standard Capital Allocation...,"ABN Amro Bank is a Dutch bank, and the Netherl...",Even after taking into considerations the more...,ABNA.XD,NaN,NaN,NaN,NaN,NaN,NaN
425,24/11/2023,Arkema Is on Track To Become a 100% Specialty ...,Arkema SA,AKE,4,09/11/2023,Rob Hales,91.06,EUR,23/11/2023,...,We assign Arkema a Standard Morningstar Capita...,Arkema is a French chemical company formerly k...,Arkema is in good financial health. Our model-...,AKEP.XD,NaN,NaN,NaN,NaN,NaN,NaN
433,24/11/2023,Nexi's Q1 Lines up With Expectations But We Lo...,Nexi SpA,NEXI,5,09/11/2023,Niklas Kammer,6.96,EUR,23/11/2023,...,We assign Nexi a Standard capital allocation r...,Nexi is a payment services provider offering m...,We anticipate that Nexi will be able to bring ...,NEXIM.XD,NaN,NaN,NaN,NaN,NaN,NaN
508,24/11/2023,Sampo Balances Underwriting Quality With Growt...,Sampo Oyj,SAMPO,3,08/11/2023,Henry Heathfield,39.65,EUR,24/11/2023,...,We think Sampo has made capital allocation dec...,Sampo is a leading Nordics-based insurer headq...,We think Sampo has a sound balance sheet. Samp...,SAMPOH.XD,NaN,NaN,NaN,NaN,NaN,NaN
586,24/11/2023,"Neoen Has an Attractive Renewables Mix, but No...",Neoen SA,NEOEN,3,NaN,Tancrede Fulop,26.66,EUR,23/11/2023,...,"Based on our capital allocation framework, whi...","Listed since 2018, Neoen is a French renewable...","Net debt was EUR 2 billion at the end of 2022,...",NEOENP.XD,NaN,NaN,NaN,NaN,NaN,NaN
603,24/11/2023,Brookfield Renewable Is Well Positioned To Tak...,Brookfield Renewable Partners LP,BEP.UN,4,03/11/2023,Brett Castelli,33.59,CAD,23/11/2023,...,We assess Brookfield Renewables capital alloca...,Brookfield Renewable is a globally diversified...,We view Brookfield Renewables financial streng...,BEP.UN,NaN,NaN,NaN,NaN,NaN,NaN
641,24/11/2023,EDP Should Deliver One of the Highest Earnings...,EDP - Energias de Portugal SA,EDP,4,03/11/2023,Tancrede Fulop,4.27,EUR,23/11/2023,...,Our Morningstar Capital Allocation Rating for ...,EDP is a vertically integrated utility company...,We forecast net debt to increase from EUR 13.6...,EDPU.XD,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(str(round(100*len(df[df["TickerPrice"].isna()])/len(df),2))+"%")

1.41%


1.23% remaining NaN is a good result!

In [4]:
# make columns price and ticker price numeric
df["Price"] = pd.to_numeric(df["Price"])
df["TickerPrice"] = pd.to_numeric(df["TickerPrice"])

In [5]:
df.dtypes

ParseDate                 object
Title                     object
CompanyName               object
TickerSymbol              object
Rating                     int64
ReportDate                object
AuthorName                object
Price                    float64
Currency                  object
PriceDate                 object
FairPrice                float64
Uncertainty               object
EconomicMoat              object
CostAllocation            object
FinancialStrength         object
AnalystNoteDate           object
AnalystNoteList           object
BullsList                 object
BearsList                 object
ResearchThesisDate        object
ResearchThesisList        object
MoatAnalysis              object
RiskAnalysis              object
CapitalAllocation         object
Profile                   object
FinancialStrengthText     object
FinalTickerSymbol         object
Sector                    object
MarketCap                float64
Exchange                  object
Volume    

In [10]:
#check whether two numbers are nearly the same
def isclose(a, b, rel_tol=8e-02, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

# Thresh
# isclose(1.05, 1.00)

# we loop over i elements in the df and check whether the price is within 5% of the price in the Price column
isclose_list = []
for i in range(len(df)):
    isclose_list.append(isclose(df["Price"][i], df["TickerPrice"][i]))
    # print progress
    print(f"{i+1}/{len(df)}")

df["IsClose"] = isclose_list


1/1626
2/1626
3/1626
4/1626
5/1626
6/1626
7/1626
8/1626
9/1626
10/1626
11/1626
12/1626
13/1626
14/1626
15/1626
16/1626
17/1626
18/1626
19/1626
20/1626
21/1626
22/1626
23/1626
24/1626
25/1626
26/1626
27/1626
28/1626
29/1626
30/1626
31/1626
32/1626
33/1626
34/1626
35/1626
36/1626
37/1626
38/1626
39/1626
40/1626
41/1626
42/1626
43/1626
44/1626
45/1626
46/1626
47/1626
48/1626
49/1626
50/1626
51/1626
52/1626
53/1626
54/1626
55/1626
56/1626
57/1626
58/1626
59/1626
60/1626
61/1626
62/1626
63/1626
64/1626
65/1626
66/1626
67/1626
68/1626
69/1626
70/1626
71/1626
72/1626
73/1626
74/1626
75/1626
76/1626
77/1626
78/1626
79/1626
80/1626
81/1626
82/1626
83/1626
84/1626
85/1626
86/1626
87/1626
88/1626
89/1626
90/1626
91/1626
92/1626
93/1626
94/1626
95/1626
96/1626
97/1626
98/1626
99/1626
100/1626
101/1626
102/1626
103/1626
104/1626
105/1626
106/1626
107/1626
108/1626
109/1626
110/1626
111/1626
112/1626
113/1626
114/1626
115/1626
116/1626
117/1626
118/1626
119/1626
120/1626
121/1626
122/1626
123/1626
1

In [27]:

#show all rows where IsClose is False
df[df["IsClose"]==False]


,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice,IsClose
57,24/11/2023,George Weston Lacks a Durable Competitive Edge...,George Weston Ltd,WN,3,21/11/2023,Dan Su,164.12,CAD,23/11/2023,...,George Weston is a holding company that contro...,We think George Weston has a solid balance she...,WN-PC.TO,Consumer Defensive,1.244874e+10,TOR,2834.0,CAD,19.240000,False
98,24/11/2023,Globant Is a High-Growth Leader in Digital Tra...,Globant SA,GLOB,3,20/11/2023,Rob Hales,212.90,USD,23/11/2023,...,Globant is a next-generation IT services compa...,Globants financial condition is strong. IT ser...,GLOBD.BA,NaN,NaN,NaN,NaN,NaN,11.450000,False
107,24/11/2023,No-Moat Snam to See Higher Allowed Returns in ...,Snam SpA,SRG,3,20/11/2023,Tancrede Fulop,4.54,EUR,23/11/2023,...,"Snam operates a network of about 38,000 kilome...",We project net debt to increase from EUR 11.9 ...,SRGM.XD,NaN,NaN,NaN,NaN,NaN,NaN,False
118,24/11/2023,NiSource Continues to Find New Investment Oppo...,NiSource Inc,NI,5,20/11/2023,Travis Miller,26.19,USD,23/11/2023,...,NiSource is one of the nation's largest natura...,NiSource has issued a substantial amount of eq...,NIMC,NaN,NaN,NaN,NaN,NaN,104.800003,False
166,24/11/2023,"Allegro's International Foray Proving Costly, ...",Allegro.EU SA,ALE,3,16/11/2023,Sean Dunlop,29.64,PLN,24/11/2023,...,Allegro is a large Polish holding company with...,We believe that Allegro's financial health is ...,0A5O.IL,Consumer Cyclical,3.055437e+10,IOB,907130.0,PLN,64.739998,False
177,24/11/2023,Allfunds: The Largest European Fund Platform,Allfunds Group PLC,ALLFG,4,16/11/2023,Niklas Kammer,5.83,EUR,23/11/2023,...,Allfunds is a fund distribution platform conne...,Allfunds is in good financial health. As of th...,ALLFGA.XD,NaN,NaN,NaN,NaN,NaN,NaN,False
226,24/11/2023,Credit and Equity Market Headwinds Will Contin...,Invesco Ltd,IVZ,3,NaN,Greggory Warren,13.85,USD,23/11/2023,...,Invesco provides investment-management service...,Invesco entered 2023 with $1.5 billion of long...,QQQ,NaN,NaN,NaN,NaN,NaN,390.059998,False
349,24/11/2023,"Credit Agricole Is a Unique, Diversified Bank ...",Credit Agricole SA,ACA,4,10/11/2023,Johann Scholtz,11.70,EUR,23/11/2023,...,Credit Agricole S.A. is majority-owned by a gr...,We estimate the average fully loaded common eq...,ACAP.XD,NaN,NaN,NaN,NaN,NaN,NaN,False
384,24/11/2023,Equity and Credit Market Headwinds Continue to...,Affiliated Managers Group Inc,AMG,3,09/11/2023,Greggory Warren,133.20,USD,23/11/2023,...,Affiliated Managers Group offers investment st...,AMG came into 2023 with $2.5 billion in princi...,MGRD,NaN,NaN,NaN,NaN,NaN,15.730000,False
413,24/11/2023,ABN Amro Well Positioned in Oligopolistic Dutc...,ABN AMRO Bank NV,ABN,5,09/11/2023,Johann Scholtz,12.38,EUR,23/11/2023,...,"ABN Amro Bank is a Dutch bank, and the Netherl...",Even after taking into considerations the more...,ABNA.XD,NaN,NaN,NaN,NaN,NaN,NaN,False


Henkel and Volkswagen should be True and BRK should be A not B (most people cant afford A so it was chosen B stock)
We manually adjust:


In [17]:
# set True in column IsClose for index 1128, 420
df["IsClose"][1128] = True
df["IsClose"][420] = True

/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46769/615948774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IsClose"][1128] = True
/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46769/615948774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IsClose"][420] = True


In [23]:
#get_market_data(["1","1","2020"], "BRK.A") for index of 237

date = df["PriceDate"][237].split("/")
symbol = "BRK-A"
price = get_market_data(date, symbol)
df["TickerPrice"][237] = price



[*********************100%%**********************]  1 of 1 completed


/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46769/1103348327.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TickerPrice"][237] = price


In [25]:
# isclose for 237 for column isClose
df["IsClose"][237] = isclose(df["Price"][237], df["TickerPrice"][237])

/var/folders/p0/qbnst5tj08g1z35zyllsm5vc0000gn/T/ipykernel_46769/3435216802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IsClose"][237] = isclose(df["Price"][237], df["TickerPrice"][237])


In [26]:
# save to csv
df.to_csv("Data/final_data.csv")

## Final market data

Now that the ticker symbols are as clean as automatically possible, we can add the market data in $n$ days from publication:

In [2]:
# load the df from csv
import pandas as pd
import numpy as np
import yfinance as yf
import time
# get date as datetime
from datetime import datetime
from datetime import timedelta
from retrying import retry
df = pd.read_csv('Data/final_data.csv', index_col=0)

In [3]:
@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_market_data(start, end, symbol):
    price = yf.download(symbol, start=start, end=end, period='1d')["Adj Close"].values[0]
    return price

This way we can get the financial data in $n$ days from the publication to test our hypothesis

In [4]:
# make a def out of this
# we need two dates because its not always trading days
def get_dates(date, n):
    date = datetime.strptime(date, '%d/%m/%Y')
    date1 = date + timedelta(days=n-1)
    date1 = date1.strftime('%Y-%m-%d')
    date2 = date + timedelta(days=n+1)
    date2 = date2.strftime('%Y-%m-%d')
    return date1, date2

In [24]:
# we use the def to get the price for the ticker symbol with our def get_market_data

ticker_prices = []
price = 0
# we loop over i elements in the df
n = 5
for i in range(len(df)):
    # we split the date into day, month and year
    date = df["PriceDate"][i]
    start, end = get_dates(date, n)
    # format as YYYY-MM-DD with datetime
    symbol = df["FinalTickerSymbol"][i]
    try:
        # get the price for the ticker symbol
        price = get_market_data(start, end, symbol)
    except:
        price = np.nan
        
    ticker_prices.append(price)
    # print progress
    print(f"{i+1}/{len(df)}")
    
df["TickerPrice5Days"] = ticker_prices

[*********************100%%**********************]  1 of 1 completed
1/1626
[*********************100%%**********************]  1 of 1 completed
2/1626
[*********************100%%**********************]  1 of 1 completed
3/1626
[*********************100%%**********************]  1 of 1 completed
4/1626
[*********************100%%**********************]  1 of 1 completed
5/1626
[*********************100%%**********************]  1 of 1 completed
6/1626
[*********************100%%**********************]  1 of 1 completed
7/1626
[*********************100%%**********************]  1 of 1 completed
8/1626
[*********************100%%**********************]  1 of 1 completed
9/1626
[*********************100%%**********************]  1 of 1 completed
10/1626
[*********************100%%**********************]  1 of 1 completed
11/1626
[*********************100%%**********************]  1 of 1 completed
12/1626
[*********************100%%**********************]  1 of 1 completed
13/1626
[*******


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-27 -> 2023-11-29)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-27 -> 2023-11-29)')



[*********************100%%**********************]  1 of 1 completed

...


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['ITXE.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-28 -> 2023-11-30)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['ITXE.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-11-28 -> 2023-11-30)')



1548/1626
[*********************100%%**********************]  1 of 1 completed
1549/1626
[*********************100%%**********************]  1 of 1 completed
1550/1626
[*********************100%%**********************]  1 of 1 completed
1551/1626
[*********************100%%**********************]  1 of 1 completed
1552/1626
[*********************100%%**********************]  1 of 1 completed
1553/1626
[*********************100%%**********************]  1 of 1 completed
1554/1626
[*********************100%%**********************]  1 of 1 completed
1555/1626
[*********************100%%**********************]  1 of 1 completed
1556/1626
[*********************100%%**********************]  1 of 1 completed
1557/1626
[*********************100%%**********************]  1 of 1 completed
1558/1626
[*********************100%%**********************]  1 of 1 completed
1559/1626
[*********************100%%**********************]  1 of 1 completed
1560/1626
[*********************100%%**************

In [33]:
# show rows where TickerPrice5Days is NaN and IsClose is True
df[(df["TickerPrice5Days"].isna()) & (df["IsClose"]==True)]

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice,IsClose,TickerPrice5Days
1530,25/11/2023,InvoCare's Scale Supports a Cost Advantage Ove...,InvoCare Ltd,IVC,3,25/09/2023,Angus Hewitt,12.67,AUD,24/11/2023,...,InvoCare's balance sheet is in a strong positi...,IVC.AX,Consumer Cyclical,1.825253e+09,ASX,2312768.0,AUD,12.67,True,NaN


Seems to be delisted since Nov 27

In [35]:
# remove all rows where TickerPrice5Days is NaN and IsClose is False
df = df[~((df["TickerPrice5Days"].isna()) | (df["IsClose"]==False))]
df



,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice,IsClose,TickerPrice5Days
0,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VMUK,4,24/11/2023,Nathan Zaia,146.75,GBP,23/11/2023,...,The capital structure and balance sheet are so...,VMUK.L,Financial Services,1.932661e+09,LSE,45564.0,GBp,157.000000,True,142.800003
1,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VUK,4,24/11/2023,Nathan Zaia,2.78,AUD,24/11/2023,...,The capital structure and balance sheet are so...,VUK.AX,Financial Services,3.709752e+09,ASX,2057263.0,AUD,2.960000,True,2.740000
2,24/11/2023,Qube's Good Momentum Likely to Carry Into Fisc...,Qube Holdings Ltd,QUB,4,24/11/2023,Adrian Atkins,2.93,AUD,24/11/2023,...,Qube is in strong financial health. Gearing (n...,QUB.AX,Industrials,5.229462e+09,ASX,2520549.0,AUD,2.930000,True,2.910000
3,24/11/2023,Stabilization of Political Risk in Spain Is Po...,Endesa SA,ELE,4,23/11/2023,Tancrede Fulop,18.84,EUR,24/11/2023,...,We expect Endesa's net debt/EBITDA ratio to de...,0N9G.IL,Utilities,2.130355e+10,IOB,5925.0,EUR,18.764191,True,19.096201
4,24/11/2023,Sage Has Reached an Inflection Point in Its Cl...,Sage Group (The) PLC,SGE,1,23/11/2023,Rob Hales,1130.50,GBP,23/11/2023,...,Sage is in good financial condition. Our model...,SGE.L,Technology,1.173311e+10,LSE,262362.0,GBp,1130.000000,True,1119.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,25/11/2023,Minimal Changes To Our Fair Value After Fiscal...,The Toronto-Dominion Bank,TD,4,NaN,Eric Compton,83.15,CAD,23/11/2023,...,We think Toronto-Dominion is in strong overall...,TD.TO,Financial Services,1.466779e+11,TOR,4963474.0,CAD,83.360001,True,83.070000
1622,25/11/2023,Xiabuxiabu's Value-Oriented Model Should Fare ...,Xiabuxiabu Catering Management (China) Hldgs C...,00520,4,28/08/2023,Ivan Su,3.25,HKD,24/11/2023,...,Xiabuxiabu holds a strong financial position w...,0520.HK,NaN,NaN,NaN,NaN,NaN,3.290000,True,3.050000
1623,25/11/2023,NextDC's Australia Wide Data Center Network Ex...,Nextdc Ltd,NXT,3,28/08/2023,Dan Baker,12.92,AUD,24/11/2023,...,"NextDC is in sound financial health, but we ex...",NXT.AX,Technology,6.918949e+09,ASX,1956904.0,AUD,13.070000,True,12.700000
1624,25/11/2023,EVT on Track To Return to Prepandemic Earnings,EVT Ltd,EVT,4,28/08/2023,Brian Han,10.82,AUD,24/11/2023,...,EVT has the liquidity to see through the curre...,EVT.AX,Communication Services,1.720115e+09,ASX,52356.0,AUD,10.830000,True,10.590000


In [36]:
# save to csv
df.to_csv("Data/5d_data.csv")

### Get market data 30d before publishing date and 30 after

In [6]:
# load the df from csv
import pandas as pd
import numpy as np
import yfinance as yf
import time
# get date as datetime
from datetime import datetime
from datetime import timedelta
from retrying import retry
import json
df = pd.read_csv('Data/final_data.csv', index_col=0)

df.head()
# pricedate, finaltickersymbol

,ParseDate,Title,CompanyName,TickerSymbol,Rating,ReportDate,AuthorName,Price,Currency,PriceDate,...,Profile,FinancialStrengthText,FinalTickerSymbol,Sector,MarketCap,Exchange,Volume,TickerCurrency,TickerPrice,IsClose
0,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VMUK,4,24/11/2023,Nathan Zaia,146.75,GBP,23/11/2023,...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VMUK.L,Financial Services,1.932661e+09,LSE,45564.0,GBp,157.000000,True
1,24/11/2023,Virgin Money Meeting Short-Term Targets but Me...,Virgin Money UK PLC,VUK,4,24/11/2023,Nathan Zaia,2.78,AUD,24/11/2023,...,Virgin Money UK was formed through the merger ...,The capital structure and balance sheet are so...,VUK.AX,Financial Services,3.709752e+09,ASX,2057263.0,AUD,2.960000,True
2,24/11/2023,Qube's Good Momentum Likely to Carry Into Fisc...,Qube Holdings Ltd,QUB,4,24/11/2023,Adrian Atkins,2.93,AUD,24/11/2023,...,Qube has three main divisions: operating; prop...,Qube is in strong financial health. Gearing (n...,QUB.AX,Industrials,5.229462e+09,ASX,2520549.0,AUD,2.930000,True
3,24/11/2023,Stabilization of Political Risk in Spain Is Po...,Endesa SA,ELE,4,23/11/2023,Tancrede Fulop,18.84,EUR,24/11/2023,...,"Endesa generates, distributes, and supplies el...",We expect Endesa's net debt/EBITDA ratio to de...,0N9G.IL,Utilities,2.130355e+10,IOB,5925.0,EUR,18.764191,True
4,24/11/2023,Sage Has Reached an Inflection Point in Its Cl...,Sage Group (The) PLC,SGE,1,23/11/2023,Rob Hales,1130.50,GBP,23/11/2023,...,The Sage Group is a U.K.-based provider of acc...,Sage is in good financial condition. Our model...,SGE.L,Technology,1.173311e+10,LSE,262362.0,GBp,1130.000000,True


In [7]:
# make a def out of this
# we need two dates because its not always trading days
def get_dates(date):
    date = datetime.strptime(date, '%d/%m/%Y')
    date1 = date - timedelta(days=31)
    date1 = date1.strftime('%Y-%m-%d')
    date2 = date + timedelta(days=31)
    date2 = date2.strftime('%Y-%m-%d')
    return date1, date2


@retry(stop_max_attempt_number=5, wait_fixed=1000)
def get_market_data(start, end, symbol):
    price = yf.download(symbol, start=start, end=end, period='1d')
    return price

In [8]:


# loop over all rows and get dates with get_dates and then get data with get_market_data
# then save to csv with the ticker symbol as name
# also save in JSON file where the key is the ticker symbol and the values are the columns of the df
# for each column the key is the date and the value is the value of the column
# this way we can easily load the data for each ticker symbol
all_data = {}
for i in range(len(df)):
    # we split the date into day, month and year
    date = df["PriceDate"][i]
    start, end = get_dates(date)
    # format as YYYY-MM-DD with datetime
    symbol = df["FinalTickerSymbol"][i]
    try:
        data = get_market_data(start, end, symbol)
        data.to_csv(f"Data/30d_data/{df['FinalTickerSymbol'][i]}.csv")
        # get the price for the ticker symbol

        # Convert data to DataFrame
        data_df = pd.DataFrame(data)
        # Convert the index to a datetime
        data_df.index = pd.to_datetime(data_df.index)
        # Convert the columns to numeric values
        for col in data_df.columns:
            data_df[col] = pd.to_numeric(data_df[col], errors='coerce')

        # Convert DataFrame to JSON
        data_json = data_df.to_json(orient='columns')
        # Add data to all_data dictionary
        all_data[symbol] = data_json

    except:
        pass
    # print progress
    print(f"{i+1}/{len(df)}")

with open('Data/30d_data/all_data.json', 'w') as f:
    json.dump(all_data, f)


[*********************100%%**********************]  1 of 1 completed
1/1626
[*********************100%%**********************]  1 of 1 completed
2/1626
[*********************100%%**********************]  1 of 1 completed
3/1626
[*********************100%%**********************]  1 of 1 completed
4/1626
[*********************100%%**********************]  1 of 1 completed
5/1626
[*********************100%%**********************]  1 of 1 completed
6/1626
[*********************100%%**********************]  1 of 1 completed
7/1626
[*********************100%%**********************]  1 of 1 completed
8/1626
[*********************100%%**********************]  1 of 1 completed
9/1626
[*********************100%%**********************]  1 of 1 completed
10/1626
[*********************100%%**********************]  1 of 1 completed
11/1626
[*********************100%%**********************]  1 of 1 completed
12/1626
[*********************100%%**********************]  1 of 1 completed
13/1626
[*******


1 Failed download:
['SRGM.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-10-23 -> 2023-12-24)')



108/1626
[*********************100%%**********************]  1 of 1 completed
109/1626
[*********************100%%**********************]  1 of 1 completed
110/1626
[*********************100%%**********************]  1 of 1 completed
111/1626
[*********************100%%**********************]  1 of 1 completed
112/1626
[*********************100%%**********************]  1 of 1 completed
113/1626
[*********************100%%**********************]  1 of 1 completed
114/1626
[*********************100%%**********************]  1 of 1 completed
115/1626
[*********************100%%**********************]  1 of 1 completed
116/1626
[*********************100%%**********************]  1 of 1 completed
117/1626
[*********************100%%**********************]  1 of 1 completed
118/1626
[*********************100%%**********************]  1 of 1 completed
119/1626
[*********************100%%**********************]  1 of 1 completed
120/1626
[*********************100%%**********************]  1 


1 Failed download:
['ALLFGA.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-10-23 -> 2023-12-24)')



178/1626
[*********************100%%**********************]  1 of 1 completed
179/1626
[*********************100%%**********************]  1 of 1 completed
180/1626
[*********************100%%**********************]  1 of 1 completed
181/1626
[*********************100%%**********************]  1 of 1 completed
182/1626
[*********************100%%**********************]  1 of 1 completed
183/1626
[*********************100%%**********************]  1 of 1 completed
184/1626
[*********************100%%**********************]  1 of 1 completed
185/1626
[*********************100%%**********************]  1 of 1 completed
186/1626
[*********************100%%**********************]  1 of 1 completed
187/1626
[*********************100%%**********************]  1 of 1 completed
188/1626
[*********************100%%**********************]  1 of 1 completed
189/1626
[*********************100%%**********************]  1 of 1 completed
190/1626
[*********************100%%**********************]  1 

...


1408/1626
[*********************100%%**********************]  1 of 1 completed
1409/1626
[*********************100%%**********************]  1 of 1 completed
1410/1626
[*********************100%%**********************]  1 of 1 completed
1411/1626
[*********************100%%**********************]  1 of 1 completed
1412/1626
[*********************100%%**********************]  1 of 1 completed
1413/1626
[*********************100%%**********************]  1 of 1 completed
1414/1626
[*********************100%%**********************]  1 of 1 completed
1415/1626
[*********************100%%**********************]  1 of 1 completed
1416/1626
[*********************100%%**********************]  1 of 1 completed
1417/1626
[*********************100%%**********************]  1 of 1 completed
1418/1626
[*********************100%%**********************]  1 of 1 completed
1419/1626
[*********************100%%**********************]  1 of 1 completed
1420/1626
[*********************100%%**************


1 Failed download:
['IVC.AX']: Exception('%ticker%: No timezone found, symbol may be delisted')



1531/1626
[*********************100%%**********************]  1 of 1 completed
1532/1626
[*********************100%%**********************]  1 of 1 completed
1533/1626
[*********************100%%**********************]  1 of 1 completed
1534/1626
[*********************100%%**********************]  1 of 1 completed
1535/1626
[*********************100%%**********************]  1 of 1 completed
1536/1626
[*********************100%%**********************]  1 of 1 completed
1537/1626
[*********************100%%**********************]  1 of 1 completed
1538/1626
[*********************100%%**********************]  1 of 1 completed
1539/1626
[*********************100%%**********************]  1 of 1 completed
1540/1626
[*********************100%%**********************]  1 of 1 completed
1541/1626
[*********************100%%**********************]  1 of 1 completed
1542/1626
[*********************100%%**********************]  1 of 1 completed
1543/1626
[*********************100%%**************


1 Failed download:
['ITXE.XD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-10-24 -> 2023-12-25)')



1548/1626
[*********************100%%**********************]  1 of 1 completed
1549/1626
[*********************100%%**********************]  1 of 1 completed
1550/1626
[*********************100%%**********************]  1 of 1 completed
1551/1626
[*********************100%%**********************]  1 of 1 completed
1552/1626
[*********************100%%**********************]  1 of 1 completed
1553/1626
[*********************100%%**********************]  1 of 1 completed
1554/1626
[*********************100%%**********************]  1 of 1 completed
1555/1626
[*********************100%%**********************]  1 of 1 completed
1556/1626
[*********************100%%**********************]  1 of 1 completed
1557/1626
[*********************100%%**********************]  1 of 1 completed
1558/1626
[*********************100%%**********************]  1 of 1 completed
1559/1626
[*********************100%%**********************]  1 of 1 completed
1560/1626
[*********************100%%**************